In [58]:
import pandas as pd

In [10]:
question 1
answer 1
answer 1.2

question 2
answer 2
answer 2.2
answer 2.3

question 3
answer 3



# the first line always is the question
# the answers starting from the next line until reach the blank line, then move to the next question
# stop when reach the end of the file
# then extract the question and answers to csv file
# firstly use pd to read the file RAG_question_vn_v1.txt

In [1]:
import pandas as pd

# Step 1: Read the file
with open('RAG_question_vn_v1.txt', 'r') as file:
    text = file.read()

# Step 2: Parse the text
segments = text.strip().split('\n\n')  # Split by blank lines
qa_pairs = []
for segment in segments:
    lines = segment.split('\n')
    question = lines[0]
    answers = lines[1:]
    # join the answers into a single string but break the line by <br> and space tag
    answers = '<br> '.join(answers)
    qa_pairs.append({'question': question, 'answers': answers})

# Step 3: Structure the data is already done in step 2

# Step 4: Convert to DataFrame
df = pd.DataFrame(qa_pairs)

# Step 5: Export to CSV
df.to_csv('questions_and_answers.csv', index=False)

In [6]:
%pip install asyncio
%pip install nest_asyncio
import pandas as pd
import nest_asyncio
import uuid 
import asyncio
import aiohttp
import json
nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [64]:
async def sync_to_qdrant_collection_qa(embedding, content):
    QdrantApi = 'http://10.13.34.154:6333/collections/'
    _points = {
        "points": [
            {
                "id": str(uuid.uuid4()), 
                "vector": embedding,
                # "payload": {"content": content},
                # treat content as json string but keep the utf-8 encoding
                "payload": {"content": json.dumps(content, ensure_ascii=False).encode('utf-8').decode('utf-8')}


            }
        ],
    }
    collection_name = 'qa_vn'
    headers = {
        "accept": "application/json, text/plain, */*",
        "pragma": "no-cache",
        "api-key": "vg@llm",
    }
    async with aiohttp.ClientSession() as session:
        async with session.put(QdrantApi + collection_name + '/points?wait=true', headers=headers, data=json.dumps(_points)) as response:
            try:
                data = await response.json()
                print('syncToQdrantCollection Success:', data)
            except Exception as error:
                print('syncToQdrantCollection Error:', error)

In [38]:


async def sync_to_qdrant_collection(embedding, content):
    QdrantApi = 'http://10.13.34.154:6333/collections/'
    _points = {
        "points": [
            {
                "id": str(uuid.uuid4()), 
                "vector": embedding,
                # "payload": {"content": content},
                # treat content as json string but keep the utf-8 encoding
                "payload": {"content": json.dumps(content, ensure_ascii=False).encode('utf-8').decode('utf-8')}


            }
        ],
    }
    collection_name = 'st_vn'
    headers = {
        "accept": "application/json, text/plain, */*",
        "pragma": "no-cache",
        "api-key": "vg@llm",
    }
    async with aiohttp.ClientSession() as session:
        async with session.put(QdrantApi + collection_name + '/points?wait=true', headers=headers, data=json.dumps(_points)) as response:
            try:
                data = await response.json()
                print('syncToQdrantCollection Success:', data)
            except Exception as error:
                print('syncToQdrantCollection Error:', error)

In [39]:

async def do_embedding_task(content):
    embedding_mdl = 'mxbai-embed-large:latest'
    embedding_api = 'http://10.13.34.154:11435/api/embeddings'
    ref = content['ref']
    knowledge = content['knowledge']
    
    async with aiohttp.ClientSession() as session:
        async with session.post(embedding_api, json={
            'model': embedding_mdl,
            'prompt': ref,
            'keep_alive': -1,
        }) as response:
            try:
                data = await response.json()
                # wait 200ms
                await asyncio.sleep(0.2)
                await sync_to_qdrant_collection(data['embedding'], knowledge)
                
            except Exception as error:
                print('doEmbeddingTask Error:', error)

In [ ]:

# Step 1: Read the file
with open('ST_VN.txt', 'r') as file:
    text = file.read()

# Step 2: Parse the text
segments = text.strip().split('\n\n')  # Split by blank lines

# the first line always is the reference address
# the answers starting from the next line until reach the blank line, then move to the next knowledge
# only one ref but multiple knowledges
# stop when reach the end of the file

# Step 3: Structure the data
contents = []
for segment in segments:
    lines = segment.split('\n')
    ref = lines[0]
    knowledges = lines[1:]
    # knowledge join as a single string
    # ending with break line as \n
    knowledge = '<br> '.join(knowledges) + '\n'
    # contents.append({'ref': ref, 'knowledge': ref + "\n" + knowledge})
    # pass the content to the do_embedding_task
    await do_embedding_task({'ref': ref, 'knowledge': ref + "\n" + knowledge})
    # wait for 200ms
    await asyncio.sleep(0.2)

# print the contents in each segment
# for content in contents:
    # print(content)

In [68]:

async def do_embedding_task_qa(content):
    embedding_mdl = 'mxbai-embed-large:latest'
    embedding_api = 'http://10.13.34.154:11435/api/embeddings'
    question = content['question']
    ref = content['ref']
    
    async with aiohttp.ClientSession() as session:
        async with session.post(embedding_api, json={
            'model': embedding_mdl,
            'prompt': question,
            'keep_alive': -1,
        }) as response:
            try:
                data = await response.json()
                # wait 200ms
                await asyncio.sleep(0.2)
                await sync_to_qdrant_collection_qa(data['embedding'], content)
                
            except Exception as error:
                print('doEmbeddingTask Error:', error)

In [69]:

# Step 1: Read the file
with open('update.txt', 'r') as file:
    text = file.read()

# Step 2: Parse the text
segments = text.strip().split('\n\n')  # Split by blank lines

# the first line always is the reference address
# the questions starting from the next line until reach the blank line, then move to the next question
# only one ref but multiple questions
# stop when reach the end of the file
# i want to use question as the input to output the ref 
# map the question to the ref

# contents = []
# my output example is [{ref: '1', q:'question 1'}, {ref: '1', q:'question 2'}]
for segment in segments:
    lines = segment.split('\n')
    ref = lines[0]
    questions = lines[1:]
    for question in questions:
        # contents.append({'q': question, 'ref': ref})
        await do_embedding_task_qa({'question': question, 'ref': ref})
        # wait for 200ms
        await asyncio.sleep(0.2)



    
                         

# print the contents in each segment
# for content in contents:
    # print(content)

syncToQdrantCollection Success: {'result': {'operation_id': 0, 'status': 'completed'}, 'status': 'ok', 'time': 0.014591885}
syncToQdrantCollection Success: {'result': {'operation_id': 1, 'status': 'completed'}, 'status': 'ok', 'time': 0.003986878}
syncToQdrantCollection Success: {'result': {'operation_id': 2, 'status': 'completed'}, 'status': 'ok', 'time': 0.012555535}
syncToQdrantCollection Success: {'result': {'operation_id': 3, 'status': 'completed'}, 'status': 'ok', 'time': 0.004812715}
syncToQdrantCollection Success: {'result': {'operation_id': 4, 'status': 'completed'}, 'status': 'ok', 'time': 0.004660557}
syncToQdrantCollection Success: {'result': {'operation_id': 5, 'status': 'completed'}, 'status': 'ok', 'time': 0.014964218}
syncToQdrantCollection Success: {'result': {'operation_id': 6, 'status': 'completed'}, 'status': 'ok', 'time': 0.013745265}
syncToQdrantCollection Success: {'result': {'operation_id': 7, 'status': 'completed'}, 'status': 'ok', 'time': 0.01345208}
syncToQdr